In [1]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import sys

In [2]:
#Loction of the dataset
folder = 'D:/Coding/Projects/Deep Learning/Dog v Cat/train/'

In [ ]:
#Plotting first 5 cat or dog images
for i in range(5):
    plt.subplot(330+1+i)
    filename = folder +'cat.' + str(i) +'.jpg'
    image = imread(filename)
    plt.imshow(image)
plt.show()

In [3]:

# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
# create directories
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'cats/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = 'train/train/'
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if random() < val_ratio:
		dst_dir = 'test/'
	if file.startswith('cat'):
		dst = dataset_home + dst_dir + 'cats/'  + file
		copyfile(src, dst)
	elif file.startswith('dog'):
		dst = dataset_home + dst_dir + 'dogs/'  + file
		copyfile(src, dst)

1. CNN

In [4]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD 
from keras.preprocessing.image import ImageDataGenerator

In [5]:
def cnn():
    model = Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same',input_shape=(200,200,3)))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten()) 
    model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
    model.add(Dense(1,activation='sigmoid'))
    opt = SGD(learning_rate=0.001,momentum=0.9)
    model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
    return model   

In [6]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	plt.subplot(212)
	plt.title('Classification Accuracy')
	plt.plot(history.history['accuracy'], color='blue', label='train')
	plt.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	
	plt.close()

# run the test harness for evaluating a model

# define model
model = cnn()
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
# prepare iterators
train_it = datagen.flow_from_directory('dataset_dogs_vs_cats/train/',
	class_mode='binary', batch_size=64, target_size=(200, 200))
test_it = datagen.flow_from_directory('dataset_dogs_vs_cats/test/',
	class_mode='binary', batch_size=64, target_size=(200, 200))
# fit model
history = model.fit(train_it, steps_per_epoch=len(train_it),
	validation_data=test_it, validation_steps=len(test_it), epochs=20)
# evaluate model
_, acc = model.evaluate(test_it, steps=len(test_it))
print('> %.3f' % (acc * 100.0))
# learning curves
#summarize_diagnostics(history)

# entry point, run the test harness


Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.
Epoch 1/20
293/293 [==============================] - 30s 92ms/step - loss: 0.6843 - accuracy: 0.5768 - val_loss: 0.6549 - val_accuracy: 0.6045
Epoch 2/20
293/293 [==============================] - 29s 100ms/step - loss: 0.6484 - accuracy: 0.6153 - val_loss: 0.7158 - val_accuracy: 0.5458
Epoch 3/20
293/293 [==============================] - 28s 97ms/step - loss: 0.6462 - accuracy: 0.6204 - val_loss: 0.6372 - val_accuracy: 0.6145
Epoch 4/20
293/293 [==============================] - 29s 98ms/step - loss: 0.6226 - accuracy: 0.6399 - val_loss: 0.6240 - val_accuracy: 0.6313
Epoch 5/20
293/293 [==============================] - 29s 98ms/step - loss: 0.6165 - accuracy: 0.6435 - val_loss: 0.6318 - val_accuracy: 0.6324
Epoch 6/20
293/293 [==============================] - 29s 98ms/step - loss: 0.6128 - accuracy: 0.6535 - val_loss: 0.6200 - val_accuracy: 0.6365
Epoch 7/20
293/293 [==============================

In [7]:
#save cnn model
model.save('dogVcatCNN.h5')